# Build Health Index

## 1. Data Collection

- Number of health services: from healthcare dataset

- Distance to centers: polygon or compute distances (3 groups or avg)

- Hospitals/CAPs: Region density over number of places offered

- Ratio of patients over doctor: dotación plantilla? cantidad personas región?

- Medical Performance:

- Deaths by region, Health characteristics of population (chronic diseases, etc.)

- Clinic Status - Pulic or Private (insurances)

- Medical Center Type Ponderation - Hospital/Pharmacy, CAP, residence/day center: from healthcare dataset

- Standarized Characteristics: specialist doctors, operating rooms ?

- Patient Satisfaction?

### Load Dataset with all Healthcare Services

In [1]:
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import chardet

def detect_encoding(file_path):
    with open(file_path, 'rb') as f:
        result = chardet.detect(f.read())
    return result['encoding']

# Usage
#encoded = detect_encoding(file_name)
#print(encoded)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [55]:
# Dataset Files
cleaned_health_file = "/content/drive/MyDrive/TFG/2. Ejecución/CODE/2.1. Data Processing/cleaned_health.csv"

encoded = detect_encoding(cleaned_health_file)
print(encoded)

# Read CSV
cleaned_health_df = pd.read_csv(cleaned_health_file, encoding='utf-8')

utf-8


In [56]:
display(cleaned_health_df)

,Unnamed: 0,register_id,Name,Street ID,Road Name,Street Number,Neighborhood ID,Neighborhood Name,District ID,District Name,ZipCode,Town,Phone Number,Service Type,geo_epgs_25831_x,geo_epgs_25831_y,Latitude,Longitude
0,0,﻿1015170605,Centre d'Atenció Primària Sardenya,76807,C Sardenya,466.0,33,el Baix Guinardó,7,Horta-Guinardó,8025.0,BARCELONA,935674380,CAP (Primary Healthcare Center),430258.038349,4.584550e+06,41.409469,2.165559
1,1,﻿68125439,Instituto Oftalmológico Tres Torres,28000,Via Augusta,281.0,24,les Tres Torres,5,Sarrià-Sant Gervasi,8017.0,BARCELONA,900842848,Hospital & Clinic,427267.424673,4.583269e+06,41.397670,2.129935
2,2,﻿75990049258,Clínica Mi Tres Torres,103502,C Doctor Roux,76.0,24,les Tres Torres,5,Sarrià-Sant Gervasi,8017.0,BARCELONA,932041300,Hospital & Clinic,427179.337491,4.583445e+06,41.399249,2.128860
3,3,﻿93056132443,Centre d'Atenció Primària Montnegre,219100,C Montnegre,21.0,19,les Corts,4,Les Corts,8029.0,BARCELONA,933632965,CAP (Primary Healthcare Center),427992.371870,4.582146e+06,41.387618,2.138740
4,4,﻿75990060288,Hospital de Barcelona,144601,Av Diagonal,660.0,19,les Corts,4,Les Corts,8034.0,BARCELONA,932542400,Hospital & Clinic,427228.518238,4.582406e+06,41.389892,2.129574
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,1571,﻿75990116588,Residència Assistencial per a Gent Gran San Pedro,309406,Rda Sant Pere,60.0,7,la Dreta de l'Eixample,2,Eixample,8010.0,BARCELONA,933102860,Residence for the elderly,431283.357815,4.582468e+06,41.390806,2.178062
1567,1572,﻿98336093115,Residència Assistida per a Gent Gran Institut ...,315500,Pg Santa Eulàlia,23.0,23,Sarrià,5,Sarrià-Sant Gervasi,8017.0,BARCELONA,932030989,Residence for the elderly,425956.699256,4.583965e+06,41.403816,2.114172
1568,1573,﻿98336115952,Residència Assistida per a Gent Gran Vibentia ...,263600,C Portell,11.0,37,el Carmel,7,Horta-Guinardó,8023.0,BARCELONA,932132131,Residence for the elderly,429053.952844,4.585498e+06,41.417910,2.151043
1569,1574,﻿99400733849,Residència Assitida per a Gent Gran La Vostra ...,137300,Carrer de la Foneria,29.0,13,la Marina de Port,3,Sants-Montjuïc,8038.0,Barcelona,NaN,Residence for the elderly,427900.987000,4.579122e+06,41.360380,2.138007


In [57]:
# Drop Unnecessary columns
cols_to_drop = ['Unnamed: 0', 'register_id', 'Street ID', 'Road Name', 'Street Number', 'Town', 'Phone Number', 'geo_epgs_25831_x', 'geo_epgs_25831_y']

cleaned_health_df.drop(columns=cols_to_drop, inplace=True)
cleaned_health_df.head(10)

,Name,Neighborhood ID,Neighborhood Name,District ID,District Name,ZipCode,Service Type,Latitude,Longitude
0,Centre d'Atenció Primària Sardenya,33,el Baix Guinardó,7,Horta-Guinardó,8025.0,CAP (Primary Healthcare Center),41.409469,2.165559
1,Instituto Oftalmológico Tres Torres,24,les Tres Torres,5,Sarrià-Sant Gervasi,8017.0,Hospital & Clinic,41.397670,2.129935
2,Clínica Mi Tres Torres,24,les Tres Torres,5,Sarrià-Sant Gervasi,8017.0,Hospital & Clinic,41.399249,2.128860
3,Centre d'Atenció Primària Montnegre,19,les Corts,4,Les Corts,8029.0,CAP (Primary Healthcare Center),41.387618,2.138740
4,Hospital de Barcelona,19,les Corts,4,Les Corts,8034.0,Hospital & Clinic,41.389892,2.129574
5,Hospital Universitari Quirón Dexeus,20,la Maternitat i Sant Ramon,4,Les Corts,8028.0,Hospital & Clinic,41.385265,2.126091
6,Centre d'Atenció Primària Ciutat Meridiana,54,Torre Baró,8,Nou Barris,8033.0,CAP (Primary Healthcare Center),41.460414,2.181428
7,Clínica Corachan,24,les Tres Torres,5,Sarrià-Sant Gervasi,8017.0,Hospital & Clinic,41.393523,2.130748
8,Clinica del Son Estivill,24,les Tres Torres,5,Sarrià-Sant Gervasi,8017.0,Hospital & Clinic,41.401672,2.128538
9,Cliníca Axisclínic *Comte d'Urgell,8,l'Antiga Esquerra de l'Eixample,2,Eixample,8036.0,Hospital & Clinic,41.386871,2.152621


### 1.1 Create Dataframe for neighborhoods

In [58]:
print("Array of Neighborhood IDs:", sorted(cleaned_health_df['Neighborhood ID'].unique()))
print("Min Neighborhood ID:", min(cleaned_health_df['Neighborhood ID'].unique()))
print("Min Neighborhood ID:", max(cleaned_health_df['Neighborhood ID'].unique()))

Array of Neighborhood IDs: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]
Min Neighborhood ID: 1
Min Neighborhood ID: 73


In [59]:
# Zone IDs
neighborhood_ids = sorted(cleaned_health_df['Neighborhood ID'].unique())
neighborhood_ids.append(42)
neighborhood_ids.sort()
print(neighborhood_ids)

# Column Names
column_names = {
    0: 'Neighborhood Name',
    1: 'Number of Services',
    2: 'Distances',
    3: 'Density over Places Ratio',
    4: 'Avg. Patients per Doctor Ratio',
    5: 'Centers Status',
    6: 'Centers Type',
    7: 'Centers Characteristics',
    8: 'Avg. Population Age',
    9: 'Population Deaths',
    10: 'Diseases',
    11: 'Territorial Socioeconomic Index (IST)',
    12: 'Avg. Gini Index (%)',
    13: 'Population Density (hab/ha)',
    14: 'Net Population Density (hab/residential ha)',
}

# Create empty dataframe with IDs as index
zones_health_df = pd.DataFrame(index=neighborhood_ids, columns=column_names.values())

# Rename Index
zones_health_df.index.name = 'Neighborhood ID'

display(zones_health_df)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73]


,Neighborhood Name,Number of Services,Distances,Density over Places Ratio,Avg. Patients per Doctor Ratio,Centers Status,Centers Type,Centers Characteristics,Avg. Population Age,Population Deaths,Diseases,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.2. Number of healthcare services

In [60]:
# Count number of rows for which Neighborhood ID holds a value
neighborhood_counts = cleaned_health_df['Neighborhood ID'].value_counts().sort_index()
#print(neighborhood_counts)
zones_health_df['Number of Services'] = neighborhood_counts
zones_health_df['Number of Services'] = zones_health_df['Number of Services'].fillna(0)
zones_health_df['Number of Services'] = zones_health_df['Number of Services'].astype(int)
zones_health_df.head(5)

,Neighborhood Name,Number of Services,Distances,Density over Places Ratio,Avg. Patients per Doctor Ratio,Centers Status,Centers Type,Centers Characteristics,Avg. Population Age,Population Deaths,Diseases,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,NaN,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.3. Neighborhood Name Mappings

In [61]:
# Neighborhood ID & Name Mapping
# Mapping Generation (Unique IDs)
neighborhood_mapping = cleaned_health_df[['Neighborhood ID', 'Neighborhood Name']].drop_duplicates()
#neighborhood_mapping = neighborhood_mapping.dropna()

# Set the index to 'Neighborhood ID'
neighborhood_mapping.set_index('Neighborhood ID', inplace=True)
# Sort IDs
neighborhood_mapping = neighborhood_mapping.sort_index()
#display(neighborhood_mapping)

# Merge Data
zones_health_df['Neighborhood Name'] = neighborhood_mapping
zones_health_df.head(5)

,Neighborhood Name,Number of Services,Distances,Density over Places Ratio,Avg. Patients per Doctor Ratio,Centers Status,Centers Type,Centers Characteristics,Avg. Population Age,Population Deaths,Diseases,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,el Barri Gòtic,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,la Barceloneta,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Sant Pere, Santa Caterina i la Ribera",26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,el Fort Pienc,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.4. IST for all Neighborhoods

In [62]:
# Dataset Files
socioec_idx_file = "/content/drive/MyDrive/TFG/2. Ejecución/CODE/IndexDatasets/IST.csv"

encoded = detect_encoding(socioec_idx_file)
print(encoded)

# Read CSV
socioec_idx_df = pd.read_csv(socioec_idx_file, encoding='utf-8', sep='\t')

utf-8


In [63]:
#display(socioec_idx_df)
# Only last year included
socioec_idx_df = socioec_idx_df[socioec_idx_df['any'] == 2020]

# Drop useless cols and renaming
socioec_idx_df = socioec_idx_df.drop(columns={'any', 'estat', 'concepte'})
socioec_idx_df.rename(columns={'barris de Barcelona': 'Neighborhood Name', 'valor': 'Territorial Socioeconomic Index (IST)'}, inplace=True)

# Reset indices
socioec_idx_df.reset_index(drop=True, inplace=True)
socioec_idx_df.index = socioec_idx_df.index + 1
socioec_idx_df.index.name = 'Neighborhood ID'
socioec_idx_df = socioec_idx_df.drop(socioec_idx_df.index[73])
display(socioec_idx_df)

,Neighborhood Name,Territorial Socioeconomic Index (IST)
Neighborhood ID,,
1,el Raval,"78,2"
2,el Barri Gòtic,"75,6"
3,la Barceloneta,"95,8"
4,"Sant Pere, Santa Caterina i la Ribera","100,8"
5,el Fort Pienc,111
...,...,...
69,Diagonal Mar i el Front Marítim del Poblenou,"116,6"
70,el Besòs i el Maresme,"72,4"
71,Provençals del Poblenou,107


In [64]:
# Compare
#socioec_idx_df['Neighborhood Name'].unique()
#zones_health_df['Neighborhood Name'].unique()

# Merge
zones_health_df['Territorial Socioeconomic Index (IST)'] = socioec_idx_df['Territorial Socioeconomic Index (IST)']
zones_health_df.head(5)

,Neighborhood Name,Number of Services,Distances,Density over Places Ratio,Avg. Patients per Doctor Ratio,Centers Status,Centers Type,Centers Characteristics,Avg. Population Age,Population Deaths,Diseases,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"78,2",NaN,NaN,NaN
2,el Barri Gòtic,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"75,6",NaN,NaN,NaN
3,la Barceloneta,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"95,8",NaN,NaN,NaN
4,"Sant Pere, Santa Caterina i la Ribera",26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"100,8",NaN,NaN,NaN
5,el Fort Pienc,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111,NaN,NaN,NaN


In [65]:
# Replace COMA with POINT for decimals in the "Territorial Socioeconomic Index (IST)" column
zones_health_df['Territorial Socioeconomic Index (IST)'] = zones_health_df['Territorial Socioeconomic Index (IST)'].str.replace(',', '.')

# Convert the column to numeric data type
zones_health_df['Territorial Socioeconomic Index (IST)'] = pd.to_numeric(zones_health_df['Territorial Socioeconomic Index (IST)'])
zones_health_df.head(5)

,Neighborhood Name,Number of Services,Distances,Density over Places Ratio,Avg. Patients per Doctor Ratio,Centers Status,Centers Type,Centers Characteristics,Avg. Population Age,Population Deaths,Diseases,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.2,NaN,NaN,NaN
2,el Barri Gòtic,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.6,NaN,NaN,NaN
3,la Barceloneta,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.8,NaN,NaN,NaN
4,"Sant Pere, Santa Caterina i la Ribera",26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.8,NaN,NaN,NaN
5,el Fort Pienc,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0,NaN,NaN,NaN


### 1.5. Gini Index(%) for all Neighborhoods

In [66]:
# Dataset Files
gini_idx_file = "/content/drive/MyDrive/TFG/2. Ejecución/CODE/IndexDatasets/2021_renda_index_gini.csv"

encoded = detect_encoding(gini_idx_file)
print(encoded)

# Read CSV
gini_idx_df = pd.read_csv(gini_idx_file, encoding='utf-8')

utf-8


In [67]:
display(gini_idx_df)

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Seccio_Censal,Index_Gini
0,2021,1,Ciutat Vella,1,el Raval,1,37.5
1,2021,1,Ciutat Vella,1,el Raval,2,35.4
2,2021,1,Ciutat Vella,1,el Raval,3,36.2
3,2021,1,Ciutat Vella,1,el Raval,4,35.0
4,2021,1,Ciutat Vella,1,el Raval,5,36.8
...,...,...,...,...,...,...,...
1063,2021,10,Sant Martí,73,la Verneda i la Pau,143,27.1
1064,2021,10,Sant Martí,65,el Clot,234,25.3
1065,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,235,32.9
1066,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,236,28.1


In [68]:
#display(socioec_idx_df)
# Drop useless cols and renaming
gini_idx_df = gini_idx_df.drop(columns={'Any', 'Codi_Districte', 'Nom_Districte'})
gini_idx_df.rename(columns={'Codi_Barri': 'Neighborhood ID', 'Nom_Barri': 'Neighborhood Name', 'Index_Gini': 'Gini Index'}, inplace=True)

display(gini_idx_df)

,Neighborhood ID,Neighborhood Name,Seccio_Censal,Gini Index
0,1,el Raval,1,37.5
1,1,el Raval,2,35.4
2,1,el Raval,3,36.2
3,1,el Raval,4,35.0
4,1,el Raval,5,36.8
...,...,...,...,...
1063,73,la Verneda i la Pau,143,27.1
1064,65,el Clot,234,25.3
1065,69,Diagonal Mar i el Front Marítim del Poblenou,235,32.9
1066,69,Diagonal Mar i el Front Marítim del Poblenou,236,28.1


In [69]:
# Compute Avg Gini Index per Neighborhood ID
avg_gini_df = gini_idx_df.groupby('Neighborhood ID')['Gini Index'].mean()

# Into sorted Dataframe by ID
avg_gini_df = avg_gini_df.reset_index()
avg_gini_df.index = avg_gini_df.index + 1
avg_gini_df.rename(columns={'Gini Index': 'Avg. Gini Index (%)'}, inplace=True)
avg_gini_df.drop(columns={'Neighborhood ID'}, inplace=True)
display(avg_gini_df)

,Avg. Gini Index (%)
1,34.495238
2,39.133333
3,33.636364
4,37.030769
5,32.475000
...,...
69,31.814286
70,29.715385
71,29.241667
72,27.361111


In [70]:
# Merge into zones dataset (round to 3 decimal places)
avg_gini_df['Avg. Gini Index (%)'] = avg_gini_df['Avg. Gini Index (%)'].round(3)
zones_health_df['Avg. Gini Index (%)'] = avg_gini_df['Avg. Gini Index (%)']
display(zones_health_df)

,Neighborhood Name,Number of Services,Distances,Density over Places Ratio,Avg. Patients per Doctor Ratio,Centers Status,Centers Type,Centers Characteristics,Avg. Population Age,Population Deaths,Diseases,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.2,34.495,NaN,NaN
2,el Barri Gòtic,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.6,39.133,NaN,NaN
3,la Barceloneta,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.8,33.636,NaN,NaN
4,"Sant Pere, Santa Caterina i la Ribera",26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.8,37.031,NaN,NaN
5,el Fort Pienc,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0,32.475,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69,Diagonal Mar i el Front Marítim del Poblenou,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,116.6,31.814,NaN,NaN
70,el Besòs i el Maresme,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.4,29.715,NaN,NaN
71,Provençals del Poblenou,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,107.0,29.242,NaN,NaN


### 1.6. Residents Density for each Neighborhood

In [71]:
# Dataset Files
density_file = "/content/drive/MyDrive/TFG/2. Ejecución/CODE/IndexDatasets/2021_densitat.csv"

encoded = detect_encoding(density_file)
print(encoded)

# Read CSV
density_df = pd.read_csv(density_file, encoding='utf-8')

utf-8


In [72]:
display(density_df)

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Població,Superfície (ha),Superfície Residencial (ha),Densitat (hab/ha),Densitat neta (hab/ha)
0,2021,1,Ciutat Vella,1,el Raval,47228,110.0,49.6,429,952
1,2021,1,Ciutat Vella,2,el Barri Gòtic,21827,81.6,34.1,268,639
2,2021,1,Ciutat Vella,3,la Barceloneta,14643,117.9,13.5,124,1081
3,2021,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",22131,111.0,32.4,199,683
4,2021,2,Eixample,5,el Fort Pienc,31262,92.9,33.6,336,930
...,...,...,...,...,...,...,...,...,...,...
68,2021,10,Sant Martí,69,Diagonal Mar i el Front Marítim del Poblenou,13323,122.7,24.0,109,556
69,2021,10,Sant Martí,70,el Besòs i el Maresme,25657,120.9,28.1,212,913
70,2021,10,Sant Martí,71,Provençals del Poblenou,20949,108.6,15.3,193,1368
71,2021,10,Sant Martí,72,Sant Martí de Provençals,26032,73.4,23.4,355,1111


In [73]:
#display(density_df)
# Drop useless cols and renaming
density_df = density_df.drop(columns={'Any', 'Codi_Districte', 'Nom_Districte', 'Nom_Barri', 'Població', 'Superfície (ha)', 'Superfície Residencial (ha)'})
density_df.rename(columns={'Codi_Barri': 'Neighborhood ID', 'Densitat (hab/ha)': 'Population Density (hab/ha)', 'Densitat neta (hab/ha)': 'Net Population Density (hab/residential ha)'}, inplace=True)

# Use Index as Neighborhood ID
density_df.drop(columns={'Neighborhood ID'}, inplace=True)
density_df.index = density_df.index + 1
display(density_df)

,Population Density (hab/ha),Net Population Density (hab/residential ha)
1,429,952
2,268,639
3,124,1081
4,199,683
5,336,930
...,...,...
69,109,556
70,212,913
71,193,1368
72,355,1111


In [74]:
# Merge data
zones_health_df['Population Density (hab/ha)'] = density_df['Population Density (hab/ha)']
zones_health_df['Net Population Density (hab/residential ha)'] = density_df['Net Population Density (hab/residential ha)']
zones_health_df.head(7)

,Neighborhood Name,Number of Services,Distances,Density over Places Ratio,Avg. Patients per Doctor Ratio,Centers Status,Centers Type,Centers Characteristics,Avg. Population Age,Population Deaths,Diseases,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,78.2,34.495,429,952
2,el Barri Gòtic,21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,75.6,39.133,268,639
3,la Barceloneta,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.8,33.636,124,1081
4,"Sant Pere, Santa Caterina i la Ribera",26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.8,37.031,199,683
5,el Fort Pienc,28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.0,32.475,336,930
6,la Sagrada Família,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111.8,32.700,493,1007
7,la Dreta de l'Eixample,133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,120.9,36.697,211,392


### 1.7. Centers Type (Service Type )

In [75]:
cleaned_health_df['Service Type'].unique()

array(['CAP (Primary Healthcare Center)', 'Hospital & Clinic',
       'Day center for the elderly', 'Residence for the elderly',
       'Pharmacy'], dtype=object)

In [76]:
# SERVICE TYPE
# Group by 'Neighborhood ID', include unique values for 'Service Type'
neighborhood_type_df = cleaned_health_df.groupby('Neighborhood ID')['Service Type'].apply(lambda x: ', '.join(x.unique()))
# Reset idx values
neighborhood_type_df = neighborhood_type_df.reset_index()
neighborhood_type_df = neighborhood_type_df[neighborhood_type_df['Neighborhood ID'] != 0]

display(neighborhood_type_df)

,Neighborhood ID,Service Type
0,1,"CAP (Primary Healthcare Center), Pharmacy, Day..."
1,2,"CAP (Primary Healthcare Center), Pharmacy, Day..."
2,3,"Hospital & Clinic, CAP (Primary Healthcare Cen..."
3,4,"CAP (Primary Healthcare Center), Pharmacy, Day..."
4,5,"CAP (Primary Healthcare Center), Pharmacy, Day..."
...,...,...
67,69,"CAP (Primary Healthcare Center), Pharmacy, Day..."
68,70,"CAP (Primary Healthcare Center), Hospital & Cl..."
69,71,"Hospital & Clinic, Pharmacy"
70,72,"CAP (Primary Healthcare Center), Pharmacy"


In [77]:
zones_health_df['Centers Type'] = neighborhood_type_df['Service Type']
zones_health_df.head(7)

,Neighborhood Name,Number of Services,Distances,Density over Places Ratio,Avg. Patients per Doctor Ratio,Centers Status,Centers Type,Centers Characteristics,Avg. Population Age,Population Deaths,Diseases,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,37,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,NaN,NaN,NaN,78.2,34.495,429,952
2,el Barri Gòtic,21,NaN,NaN,NaN,NaN,"Hospital & Clinic, CAP (Primary Healthcare Cen...",NaN,NaN,NaN,NaN,75.6,39.133,268,639
3,la Barceloneta,13,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,NaN,NaN,NaN,95.8,33.636,124,1081
4,"Sant Pere, Santa Caterina i la Ribera",26,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,NaN,NaN,NaN,100.8,37.031,199,683
5,el Fort Pienc,28,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,NaN,NaN,NaN,111.0,32.475,336,930
6,la Sagrada Família,37,NaN,NaN,NaN,NaN,"Hospital & Clinic, CAP (Primary Healthcare Cen...",NaN,NaN,NaN,NaN,111.8,32.700,493,1007
7,la Dreta de l'Eixample,133,NaN,NaN,NaN,NaN,"Hospital & Clinic, CAP (Primary Healthcare Cen...",NaN,NaN,NaN,NaN,120.9,36.697,211,392


### 1.8. Average Neighbourhood Population Age

In [78]:
# Dataset Files
age_file = "/content/drive/MyDrive/TFG/2. Ejecución/CODE/HealthDatasets/2023_edat.csv"

encoded = detect_encoding(age_file)
print(encoded)

# Read CSV
age_df = pd.read_csv(age_file, encoding='utf-8')

utf-8


In [79]:

age_df.drop(columns={'Data_Referencia', 'AEB', 'Seccio_Censal'}, inplace=True)
age_df.rename(columns={'Codi_Districte': 'District ID', 'Nom_Districte': 'District Name', 'Codi_Barri': 'Neighborhood ID',
                       'Nom_Barri': 'Neighborhood Name', 'Valor': 'Amount', 'EDAT_1': 'Age'}, inplace=True)
age_df.head()

,District ID,District Name,Neighborhood ID,Neighborhood Name,Amount,Age
0,1,Ciutat Vella,1,el Raval,9,0
1,1,Ciutat Vella,1,el Raval,15,1
2,1,Ciutat Vella,1,el Raval,11,2
3,1,Ciutat Vella,1,el Raval,11,3
4,1,Ciutat Vella,1,el Raval,14,4


In [80]:
age_df = age_df[age_df['Amount'] != '..']

In [81]:
age_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 94990 entries, 0 to 104939
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   District ID        94990 non-null  int64 
 1   District Name      94990 non-null  object
 2   Neighborhood ID    94990 non-null  int64 
 3   Neighborhood Name  94990 non-null  object
 4   Amount             94990 non-null  object
 5   Age                94990 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 5.1+ MB


In [82]:
age_df['Amount'] = pd.to_numeric(age_df['Amount'])

In [83]:
def calculate_weighted_average(group):
  group['Weighted_Age'] = group['Amount'] * group['Age']
  weighted_sum = group['Weighted_Age'].sum()
  sum_valor = group['Amount'].sum()
  weighted_average_age = weighted_sum / sum_valor
  return pd.Series({'Weighted_Average_Age': weighted_average_age})

In [84]:
weighted_averages = age_df.groupby(['Neighborhood Name', 'Neighborhood ID']).apply(calculate_weighted_average)
weighted_averages.rename(columns={'Weighted_Average_Age': 'Avg. Population Age'}, inplace=True)
weighted_averages['Avg. Population Age'] = weighted_averages['Avg. Population Age'].round(3)
weighted_averages.head()

,,Avg. Population Age
Neighborhood Name,Neighborhood ID,
Baró de Viver,58,40.104
Can Baró,34,43.576
Can Peguera,47,44.954
Canyelles,49,48.552
Ciutat Meridiana,55,39.408


In [85]:
# Sort by neighborhood IDs
# Single Index
weighted_averages = weighted_averages.reset_index(level=0, drop=True)
#print(zones_health_df.index)
#print(weighted_averages.index)

# Reset index of weighted_averages to extract 'Neighborhood ID'
weighted_averages.sort_values("Neighborhood ID", ascending=True, inplace=True)


# Assign Value
zones_health_df['Avg. Population Age'] = weighted_averages['Avg. Population Age']

In [86]:
zones_health_df.head()

,Neighborhood Name,Number of Services,Distances,Density over Places Ratio,Avg. Patients per Doctor Ratio,Centers Status,Centers Type,Centers Characteristics,Avg. Population Age,Population Deaths,Diseases,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,37,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,37.868,NaN,NaN,78.2,34.495,429,952
2,el Barri Gòtic,21,NaN,NaN,NaN,NaN,"Hospital & Clinic, CAP (Primary Healthcare Cen...",NaN,37.769,NaN,NaN,75.6,39.133,268,639
3,la Barceloneta,13,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,42.427,NaN,NaN,95.8,33.636,124,1081
4,"Sant Pere, Santa Caterina i la Ribera",26,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,40.082,NaN,NaN,100.8,37.031,199,683
5,el Fort Pienc,28,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,43.307,NaN,NaN,111.0,32.475,336,930


### 1.9. Population Deaths

In [87]:
# Dataset Files
deaths_file = "/content/drive/MyDrive/TFG/2. Ejecución/CODE/HealthDatasets/2022_def_nacionalitat.csv"

encoded = detect_encoding(deaths_file)
print(encoded)

# Read CSV
deaths_df = pd.read_csv(deaths_file, encoding='utf-8')

utf-8


In [88]:
deaths_df.drop(columns={'Any', 'AEB', 'Seccio_Censal', 'NACIONALITAT_G'}, inplace=True)
deaths_df.rename(columns={'Codi_Districte': 'District ID', 'Nom_Districte': 'District Name', 'Codi_Barri': 'Neighborhood ID',
                       'Nom_Barri': 'Neighborhood Name', 'Valor': 'Amount'}, inplace=True)
deaths_df.head()

,District ID,District Name,Neighborhood ID,Neighborhood Name,Amount
0,1,Ciutat Vella,1,el Raval,7
1,1,Ciutat Vella,1,el Raval,..
2,1,Ciutat Vella,1,el Raval,15
3,1,Ciutat Vella,1,el Raval,..
4,1,Ciutat Vella,1,el Raval,18


In [89]:
deaths_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1438 entries, 0 to 1437
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   District ID        1438 non-null   int64 
 1   District Name      1438 non-null   object
 2   Neighborhood ID    1438 non-null   int64 
 3   Neighborhood Name  1438 non-null   object
 4   Amount             1438 non-null   object
dtypes: int64(2), object(3)
memory usage: 56.3+ KB


In [90]:
deaths_df = deaths_df[deaths_df['Amount'] != '..']
deaths_df['Amount'] = pd.to_numeric(deaths_df['Amount'])

In [91]:
deaths_amount_df = deaths_df.groupby(['Neighborhood Name', 'Neighborhood ID']).sum()
deaths_amount_df.sort_values("Neighborhood ID", ascending=True, inplace=True)
deaths_amount_df.head()


,,District ID,District Name,Amount
Neighborhood Name,Neighborhood ID,,,
el Raval,1,21,Ciutat VellaCiutat VellaCiutat VellaCiutat Vel...,270
el Barri Gòtic,2,10,Ciutat VellaCiutat VellaCiutat VellaCiutat Vel...,139
la Barceloneta,3,10,Ciutat VellaCiutat VellaCiutat VellaCiutat Vel...,127
"Sant Pere, Santa Caterina i la Ribera",4,13,Ciutat VellaCiutat VellaCiutat VellaCiutat Vel...,137
el Fort Pienc,5,42,EixampleEixampleEixampleEixampleEixampleEixamp...,345


In [94]:
# Single index
deaths_amount_df = deaths_amount_df.reset_index(level=0, drop=True)
print(zones_health_df.index)
print(deaths_amount_df.index)

# Assign Value
zones_health_df['Population Deaths'] = deaths_amount_df['Amount']
zones_health_df.head(7)

Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54,
       55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72,
       73],
      dtype='int64', name='Neighborhood ID')
RangeIndex(start=0, stop=72, step=1)


,Neighborhood Name,Number of Services,Distances,Density over Places Ratio,Avg. Patients per Doctor Ratio,Centers Status,Centers Type,Centers Characteristics,Avg. Population Age,Population Deaths,Diseases,Territorial Socioeconomic Index (IST),Avg. Gini Index (%),Population Density (hab/ha),Net Population Density (hab/residential ha)
Neighborhood ID,,,,,,,,,,,,,,,
1,el Raval,37,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,37.868,139.0,NaN,78.2,34.495,429,952
2,el Barri Gòtic,21,NaN,NaN,NaN,NaN,"Hospital & Clinic, CAP (Primary Healthcare Cen...",NaN,37.769,127.0,NaN,75.6,39.133,268,639
3,la Barceloneta,13,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,42.427,137.0,NaN,95.8,33.636,124,1081
4,"Sant Pere, Santa Caterina i la Ribera",26,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,40.082,345.0,NaN,100.8,37.031,199,683
5,el Fort Pienc,28,NaN,NaN,NaN,NaN,"CAP (Primary Healthcare Center), Pharmacy, Day...",NaN,43.307,503.0,NaN,111.0,32.475,336,930
6,la Sagrada Família,37,NaN,NaN,NaN,NaN,"Hospital & Clinic, CAP (Primary Healthcare Cen...",NaN,44.562,462.0,NaN,111.8,32.700,493,1007
7,la Dreta de l'Eixample,133,NaN,NaN,NaN,NaN,"Hospital & Clinic, CAP (Primary Healthcare Cen...",NaN,43.213,353.0,NaN,120.9,36.697,211,392


### 1.10.